# Haptic Classification Training

This notebook creates the Haptic Network Architecture and trains it for grasp testset1. After training the network weights will be stored in the folder `grasp_Freq_50TimePerSec_7channels_testset1_logs`

## Dependencies

`Python 3.5.4` is used for development and following packages are required to run the code provided in the notebook:

`pip install googledrivedownloader`<br>
`pip install matplotlib`<br>
`pip install tensorflow-gpu`<br>
`pip install numpy`

In [1]:
import pickle, os, csv, time, shutil
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd

In [2]:
def print_image(image, title):
    """Print the image

    :param image: image pixels in list
    :param title: title as string to be printed on top of the image
    """
    plt.imshow(image)
    plt.title(title)
    plt.colorbar()
    plt.show()

def time_taken(start, end):
    """Human readable time between `start` and `end`

    :param start: time.time()
    :param end: time.time()
    :returns: day:hour:minute:second
    """
    time = end-start
    day = time // (24 * 3600)
    time = time % (24 * 3600)
    hour = time // 3600
    time %= 3600
    minutes = time // 60
    time %= 60
    seconds = time
    day_hour_min_sec = str('%02d' % int(day))+":"+str('%02d' % int(hour))+":"+str('%02d' % int(minutes))+":"+str('%02d' % int(seconds))
    
    return day_hour_min_sec

In [3]:
"""
Downloading `category_object_labels.bin`.
This contains two dictionaries that stores category labels and object labels
"""

gdd.download_file_from_google_drive(file_id='15xPAbso4-uLk8PcRi9cb70r1r3FaU6ux',
                                    dest_path='./dataset/category_object_labels.bin',
                                    unzip=False)

bin_file = open("./dataset/category_object_labels.bin", "rb")

category_labels = pickle.load(bin_file)
object_labels = pickle.load(bin_file)

bin_file.close()

## Haptic Dataset

In [4]:
"""
Downloading testset1 for haptic of grasp interaction and reading it.
"""

db_file_name = "grasp_Freq_50TimePerSec_7channels_testset1.bin"
gdd.download_file_from_google_drive(file_id='16g4Cp67SHqpVSUyOryPEH2HCTZhMcCeB',
                                    dest_path='./dataset/'+db_file_name,
                                    unzip=False)

bin_file = open("./dataset/"+db_file_name, "rb")

haptic_frames_2d_train = pickle.load(bin_file)
category_label_train = pickle.load(bin_file)
object_label_train = pickle.load(bin_file)
category_label_train_one_not = pickle.load(bin_file)
object_label_train_one_not = pickle.load(bin_file)

haptic_frames_2d_test = pickle.load(bin_file)
category_label_test = pickle.load(bin_file)
object_label_test = pickle.load(bin_file)
category_label_test_one_not = pickle.load(bin_file)
object_label_test_one_not = pickle.load(bin_file)

bin_file.close()

In [5]:
"""
Printing shape of the haptic dataset
"""

batch_no = 188

print("batch, frames, channel: ", haptic_frames_2d_train.shape)

print("Category: ", list(category_labels.keys())[list(category_labels.values()).index(category_label_train[batch_no])])
print("Object: ", list(object_labels.keys())[list(object_labels.values()).index(object_label_train[batch_no])])

batch, frames, channel:  (400, 100, 7)
Category:  egg
Object:  egg_rough_styrofoam


## Building the Haptic Network Architecture

![Haptic Network Architecture](pics/Haptic_CNN.png)

In [6]:
# Network hyper-parameters
batch = 5
training_epochs = 400
display_step = 1

frames = time_step_size = haptic_frames_2d_train.shape[1]
channel = haptic_frames_2d_train.shape[2]

num_classes = category_label_train_one_not.shape[1]

# Network Parameters
learning_rate = 0.0001

model_path = db_file_name.split(".")[0]+"_logs/model.ckpt"
logs_path = db_file_name.split(".")[0]+"_logs/"

X = tf.placeholder('float', [None, frames, channel], name='InputData')
Y = tf.placeholder('float', [None, num_classes], name='LabelData')
keep_prob = tf.placeholder_with_default(1.0, shape=(), name = 'keep')

In [7]:
skip_2nd_maxpool = ["grasp", "hold", "low"]

# Create model
def model():
    with tf.name_scope("Model"):
        data_placeholder = tf.placeholder('float', [None, frames, channel], name='InputData')
        
        net = tf.reshape(data_placeholder, [-1, frames, channel, 1])
        net = tf.layers.conv2d(inputs=net, filters=32, kernel_size=[20, 5], padding="same", activation=tf.nn.relu)
        net = tf.layers.max_pooling2d(inputs=net, pool_size=[10, 1], strides=2)
        net = tf.layers.conv2d(inputs=net, filters=64, kernel_size=[1, 3], padding="same", activation=tf.nn.relu)
        if (db_file_name.split("_")[0]) in skip_2nd_maxpool:
            net = tf.layers.max_pooling2d(inputs=net, pool_size=[1, 1], strides=[1, 2])
        else:
            net = tf.layers.max_pooling2d(inputs=net, pool_size=[10, 1], strides=2)
        net = tf.layers.flatten(net)
        # Dense Layer
        net = tf.layers.dense(inputs=net, units=1024, activation=tf.nn.relu)
        # Add dropout operation
        net = tf.layers.dropout(inputs=net, rate=keep_prob)
        logits = tf.layers.dense(inputs=net, units=num_classes)
        
    return logits

def loss(prediction, label_placeholder):
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=label_placeholder))
        # Create a summary to monitor cost tensor
        cost_scalar = tf.summary.scalar("loss", cost)
    return cost, cost_scalar

def training(prediction, label_placeholder):
    with tf.name_scope('Optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(cost)
    return train_op

def evaluate(prediction, Y):
    with tf.name_scope('Accuracy'):
        # Test model
        correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
        # Create a summary to monitor accuracy tensor
        accuracy_scalar = tf.summary.scalar("accuracy", accuracy)
    return accuracy, accuracy_scalar

In [8]:
"""
Creating the Neural Network
"""

model_dict = {}

prediction = model()
model_dict["Model"] = prediction

cost, cost_scalar = loss(prediction, Y)
model_dict["Loss"] = cost
model_dict["Loss_scalar"] = cost_scalar

train_op = training(prediction, Y)
model_dict["Optimizer"] = train_op

eval_op, accuracy_scalar = evaluate(prediction, Y)
model_dict["Accuracy"] = eval_op
model_dict["Accuracy_scalar"] = accuracy_scalar

print("model_dict: ", model_dict)

# Initializing the variables
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver(max_to_keep=1)

model_dict:  {'Model': <tf.Tensor 'Model/dense_1/BiasAdd:0' shape=(?, 20) dtype=float32>, 'Loss': <tf.Tensor 'Loss/Mean:0' shape=() dtype=float32>, 'Loss_scalar': <tf.Tensor 'Loss/loss:0' shape=() dtype=string>, 'Optimizer': <tf.Operation 'Optimizer/Adam' type=NoOp>, 'Accuracy': <tf.Tensor 'Accuracy/Mean:0' shape=() dtype=float32>, 'Accuracy_scalar': <tf.Tensor 'Accuracy/accuracy:0' shape=() dtype=string>}


In [9]:
if os.path.exists(logs_path):
    shutil.rmtree(logs_path)
    os.makedirs(logs_path)
else:
    os.makedirs(logs_path)

In [10]:
"""
Writing 'Time', 'Epoch', 'Cost', 'Accuracy' in CSV file
"""

epoch_cost_accuracy = []
epoch_cost_accuracy.append("Time")
epoch_cost_accuracy.append("Epoch")
epoch_cost_accuracy.append("Cost")
epoch_cost_accuracy.append("Accuracy")

with open(logs_path+db_file_name.split(".")[0]+"_data.csv",'w') as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerow(epoch_cost_accuracy)

## Training

In [11]:
# Start Training

start_time = time.time()

with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost_list = 0.0
        total_batch = int(len(haptic_frames_2d_train)/batch)
        
        # Shuffle data
        shuffle_indices = np.arange(len(haptic_frames_2d_train))
        np.random.shuffle(shuffle_indices)
        
        i = 0
        # Loop over all batches
        for start, end in zip(range(0, len(haptic_frames_2d_train), batch), range(batch, len(haptic_frames_2d_train)+1, batch)):
            input_data, label_data = haptic_frames_2d_train[shuffle_indices][start:end], category_label_train_one_not[shuffle_indices][start:end]
            X = tf.get_default_graph().get_tensor_by_name("Model/InputData:0")
            _, new_cost, cost_scalar = sess.run([model_dict["Optimizer"], model_dict["Loss"], model_dict["Loss_scalar"]], feed_dict={X: input_data, Y: label_data})
            # Compute average loss
            avg_cost_list += new_cost/total_batch
            summary_writer.add_summary(cost_scalar, epoch * total_batch + i)

            i += 1
            
        save_path = saver.save(sess, model_path, epoch)
         
        # Calculate Accuracy
        avg_accuracy_list = 0.0
        total_batch = int(len(haptic_frames_2d_test)/batch)
        i = 0
        for start, end in zip(range(0, len(haptic_frames_2d_test), batch), range(batch, len(haptic_frames_2d_test)+1, batch)):
            input_data, label_data = haptic_frames_2d_test[start:end], category_label_test_one_not[start:end]
            X = tf.get_default_graph().get_tensor_by_name("Model/InputData:0")
            accuracy, accuracy_scalar = sess.run([model_dict["Accuracy"], model_dict["Accuracy_scalar"]], feed_dict={X: input_data, Y: label_data})
            # Compute average accuracy
            avg_accuracy_list += accuracy/total_batch
            summary_writer.add_summary(accuracy_scalar, epoch * total_batch + i)
            
            i += 1
        
        # Printing current epoch accuracy
        epoch_cost_accuracy = []
        epoch_cost_accuracy.append(time_taken(start_time, time.time()))
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), ", Time: ", time_taken(start_time, time.time()))
            a_string = "Cost - "
            epoch_cost_accuracy.append(epoch+1)
            
            a_string += str(avg_cost_list)
            epoch_cost_accuracy.append(str(avg_cost_list))
            
            a_string = a_string[0:-2]+" --> Accuracy - "
            a_string += str(avg_accuracy_list)
            epoch_cost_accuracy.append(str(avg_accuracy_list))
            
            print(a_string)
        
        # Writing current epoch data
        with open(logs_path+db_file_name.split(".")[0]+"_data.csv", 'a') as f: # append to the file created
            writer = csv.writer(f, lineterminator="\n")
            writer.writerow(epoch_cost_accuracy)
    
    print("Optimization Finished!")
    end_time = time.time()
    print("Time taken: day, hour, minutes, seconds->", time_taken(start_time, end_time))

Epoch: 0001 , Time:  00:00:00:04
Cost - 2.9825677961111 --> Accuracy - 0.15000000298023225
Epoch: 0002 , Time:  00:00:00:06
Cost - 2.7820644050836 --> Accuracy - 0.18000000491738322
Epoch: 0003 , Time:  00:00:00:08
Cost - 2.5925866603851 --> Accuracy - 0.20000000447034838
Epoch: 0004 , Time:  00:00:00:10
Cost - 2.42690599113702 --> Accuracy - 0.25000000521540644
Epoch: 0005 , Time:  00:00:00:12
Cost - 2.3041903585195 --> Accuracy - 0.27000000402331353
Epoch: 0006 , Time:  00:00:00:14
Cost - 2.16228994578123 --> Accuracy - 0.1800000049173832
Epoch: 0007 , Time:  00:00:00:16
Cost - 2.10475341230630 --> Accuracy - 0.28000000566244126
Epoch: 0008 , Time:  00:00:00:18
Cost - 2.02683709263801 --> Accuracy - 0.2500000052154064
Epoch: 0009 , Time:  00:00:00:20
Cost - 1.92295539528131 --> Accuracy - 0.3300000078976154
Epoch: 0010 , Time:  00:00:00:22
Cost - 1.8511080071330 --> Accuracy - 0.3200000040233135
Epoch: 0011 , Time:  00:00:00:24
Cost - 1.7919423401355 --> Accuracy - 0.3200000062584877

Epoch: 0091 , Time:  00:00:03:04
Cost - 0.145208079670555 --> Accuracy - 0.3800000071525575
Epoch: 0092 , Time:  00:00:03:06
Cost - 0.165558858186705 --> Accuracy - 0.37000000774860387
Epoch: 0093 , Time:  00:00:03:08
Cost - 0.141749881068244 --> Accuracy - 0.35000000670552256
Epoch: 0094 , Time:  00:00:03:10
Cost - 0.169948584306985 --> Accuracy - 0.3600000083446503
Epoch: 0095 , Time:  00:00:03:12
Cost - 0.15145254991366 --> Accuracy - 0.3800000086426736
Epoch: 0096 , Time:  00:00:03:14
Cost - 0.156915412441594 --> Accuracy - 0.37000000774860387
Epoch: 0097 , Time:  00:00:03:16
Cost - 0.13052815099363 --> Accuracy - 0.3600000075995923
Epoch: 0098 , Time:  00:00:03:18
Cost - 0.141537678439635 --> Accuracy - 0.35000000745058063
Epoch: 0099 , Time:  00:00:03:20
Cost - 0.207786356826545 --> Accuracy - 0.38000000715255744
Epoch: 0100 , Time:  00:00:03:22
Cost - 0.16128903574426 --> Accuracy - 0.39000000804662716
Epoch: 0101 , Time:  00:00:03:24
Cost - 0.161709482956211 --> Accuracy - 0.40

Epoch: 0180 , Time:  00:00:06:06
Cost - 0.197492799954488 --> Accuracy - 0.33000000715255745
Epoch: 0181 , Time:  00:00:06:08
Cost - 0.28132471534627 --> Accuracy - 0.32000000476837165
Epoch: 0182 , Time:  00:00:06:10
Cost - 0.120967188421491 --> Accuracy - 0.3300000071525574
Epoch: 0183 , Time:  00:00:06:12
Cost - 0.0536994707726989 --> Accuracy - 0.3300000064074993
Epoch: 0184 , Time:  00:00:06:14
Cost - 0.04529439862235 --> Accuracy - 0.3500000067055226
Epoch: 0185 , Time:  00:00:06:16
Cost - 0.0118635222708689 --> Accuracy - 0.35000000745058063
Epoch: 0186 , Time:  00:00:06:18
Cost - 0.0093686056832666 --> Accuracy - 0.35000000745058063
Epoch: 0187 , Time:  00:00:06:21
Cost - 0.008304375702573 --> Accuracy - 0.3600000075995923
Epoch: 0188 , Time:  00:00:06:23
Cost - 0.0077886630642751 --> Accuracy - 0.3600000075995922
Epoch: 0189 , Time:  00:00:06:25
Cost - 0.00746612610091688 --> Accuracy - 0.3600000075995922
Epoch: 0190 , Time:  00:00:06:27
Cost - 0.00699929603761 --> Accuracy - 

Epoch: 0268 , Time:  00:00:09:15
Cost - 0.00269423361914 --> Accuracy - 0.34000000804662706
Epoch: 0269 , Time:  00:00:09:17
Cost - 0.0025450825942243 --> Accuracy - 0.34000000804662706
Epoch: 0270 , Time:  00:00:09:19
Cost - 0.0024510590938007 --> Accuracy - 0.35000000745058063
Epoch: 0271 , Time:  00:00:09:21
Cost - 0.0024144575577338 --> Accuracy - 0.3600000083446503
Epoch: 0272 , Time:  00:00:09:24
Cost - 0.00230415125315630 --> Accuracy - 0.3600000083446503
Epoch: 0273 , Time:  00:00:09:26
Cost - 0.0022059670522139 --> Accuracy - 0.34000000804662706
Epoch: 0274 , Time:  00:00:09:28
Cost - 0.0021946346914774 --> Accuracy - 0.37000000849366194
Epoch: 0275 , Time:  00:00:09:30
Cost - 0.00207166080435854 --> Accuracy - 0.36000000908970836
Epoch: 0276 , Time:  00:00:09:32
Cost - 0.00197604470176884 --> Accuracy - 0.3500000089406967
Epoch: 0277 , Time:  00:00:09:34
Cost - 0.00199267351090384 --> Accuracy - 0.36000000908970836
Epoch: 0278 , Time:  00:00:09:36
Cost - 0.00187951947045803 -

Epoch: 0356 , Time:  00:00:12:22
Cost - 0.00070816836246194 --> Accuracy - 0.38000000715255744
Epoch: 0357 , Time:  00:00:12:24
Cost - 0.00069148248753663 --> Accuracy - 0.38000000715255744
Epoch: 0358 , Time:  00:00:12:26
Cost - 0.00065936980713559 --> Accuracy - 0.38000000715255744
Epoch: 0359 , Time:  00:00:12:28
Cost - 0.00064602962984281 --> Accuracy - 0.37000000774860387
Epoch: 0360 , Time:  00:00:12:30
Cost - 0.00062830074778759 --> Accuracy - 0.38000000715255744
Epoch: 0361 , Time:  00:00:12:32
Cost - 0.00061117545360502 --> Accuracy - 0.38000000715255744
Epoch: 0362 , Time:  00:00:12:35
Cost - 0.00058500331174400 --> Accuracy - 0.37000000774860387
Epoch: 0363 , Time:  00:00:12:37
Cost - 0.00056326555636587 --> Accuracy - 0.38000000715255744
Epoch: 0364 , Time:  00:00:12:39
Cost - 0.00055161616710392 --> Accuracy - 0.37000000774860387
Epoch: 0365 , Time:  00:00:12:41
Cost - 0.00055538244705530 --> Accuracy - 0.38000000715255744
Epoch: 0366 , Time:  00:00:12:43
Cost - 0.00052344